In [1]:
import time
import numpy as np
import pandas as pd
import sys
# sys.path.append("Finta Indicators")
sys.path.append(r"C:\Users\user\Dropbox\+Trading\AlgorithmicTrading\AlgoTrading\core\feature_extraction")
from finta_me import FinTA
# from indicators_me import *

%config Completer.use_jedi = False

In [3]:
# data = pd.read_csv("BTC_XRP_5min.csv", index_col=0).tail(200000)
data = pd.read_csv("../Data/Daily/US_Market/latest/AAPL_USD_20000101_to_20211124_daily_yahoo_finance.csv", parse_dates=True, 
                   index_col=0).tail(1000)
data = data.rename(columns={"Open": "open", 'Close': "close", "High": "high", "Low": "low", "Volume":"volume"})
print(data.shape)
data.head(3)


(1000, 6)


,open,high,low,close,Adj Close,volume
Date,,,,,,
2017-12-06,41.875000,42.549999,41.615002,42.252499,40.411076,114240000
2017-12-07,42.257500,42.610001,42.227501,42.330002,40.485199,102693200
2017-12-08,42.622501,42.750000,42.205002,42.342499,40.497154,93420800


### Using Finta, check what features are Value (Price) dependent

In [4]:
data2 = (data / 10).reset_index()
data = data.reset_index()#.drop('Date', axis=1)

In [10]:
indicators_finta = pd.read_csv("Finta Indicators/indicators_finta.csv").values
# indicators_finta_symbols = indicators_finta[:,1]
indicators_finta[:3]

array([['Simple Moving Average', 'SMA'],
       ['Simple Moving Median', 'SMM'],
       ['Smoothed Simple Moving Average', 'SSMA']], dtype=object)

In [6]:
### ------ Not implemented indicators --------------
# method_list = [func for func in dir(TA) if callable(getattr(TA, func)) and not func.startswith("__")]
# print(len(method_list))
# set(method_list) - set(indicators_finta_symbols)

In [7]:
# indicators_finta = [
#            ['Moving Average Convergence Divergence', 'MACD'],
# #            ['Volume-Weighted MACD', 'VW_MACD'],
# #            ['Elastic-Volume weighted MACD', 'EV_MACD'],
# ]

In [12]:
%%time
# Check if they are Value dependent
indicators_analysis = []
for indicator in indicators_finta[:]:
#     print(indicator, "\t")
    st = time.time()
    res1 = eval(f"FinTA.{indicator[1]}(data)")   # , *{args}
    res2 = eval(f"FinTA.{indicator[1]}(data2)")  # , *{args}
    
    value_dependant = 'No' if (np.allclose(res1.values, res2.values,  atol=1e-2, equal_nan=True )) else 'Yes'
    
    elapsed_time = time.time() - st
    
    indicators_analysis.append([indicator[0], indicator[1], value_dependant, elapsed_time])
indicators_analysis = pd.DataFrame(indicators_analysis, columns=('Indicator', 'Symbol',"Value dependent", "Processing time"))
indicators_analysis.head(20)

Wall time: 12.6 s


,Indicator,Symbol,Value dependent,Processing time
0,Simple Moving Average,SMA,Yes,0.004994
1,Simple Moving Median,SMM,Yes,0.007995
2,Smoothed Simple Moving Average,SSMA,Yes,0.003998
3,Exponential Moving Average,EMA,Yes,0.003998
4,Double Exponential Moving Average,DEMA,Yes,0.014991
5,Triple Exponential Moving Average,TEMA,Yes,0.017994
6,Triangular Moving Average,TRIMA,Yes,0.006991
7,Triple Exponential Moving Average Oscillator,TRIX,No,0.008995
8,Volume Adjusted Moving Average,VAMA,Yes,0.004997
9,Kaufman Efficiency Indicator,ER,No,0.004997


#### price_dependent_indicators

In [15]:
price_dependent_indicators = indicators_analysis[indicators_analysis['Value dependent']== 'Yes']
# price_independent_indicators[['Indicator', 'Symbol']].to_csv("Technical_indicators.csv", index=False)
price_dependent_indicators.head(70)

,Indicator,Symbol,Value dependent,Processing time
0,Simple Moving Average,SMA,Yes,0.004994
1,Simple Moving Median,SMM,Yes,0.007995
2,Smoothed Simple Moving Average,SSMA,Yes,0.003998
3,Exponential Moving Average,EMA,Yes,0.003998
4,Double Exponential Moving Average,DEMA,Yes,0.014991
5,Triple Exponential Moving Average,TEMA,Yes,0.017994
6,Triangular Moving Average,TRIMA,Yes,0.006991
8,Volume Adjusted Moving Average,VAMA,Yes,0.004997
10,Kaufman s Adaptive Moving Average,KAMA,Yes,0.013014
11,Zero Lag Exponential Moving Average,ZLEMA,Yes,0.002977


#### price independent_indicators

In [14]:
price_independent_indicators = indicators_analysis[indicators_analysis['Value dependent']== 'No']
# price_independent_indicators[['Indicator', 'Symbol']].to_csv("Technical_indicators.csv", index=False)
price_independent_indicators.head(70)

,Indicator,Symbol,Value dependent,Processing time
7,Triple Exponential Moving Average Oscillator,TRIX,No,0.008995
9,Kaufman Efficiency Indicator,ER,No,0.004997
19,Percentage Price Oscillator,PPO,No,0.006977
23,Rate-of-Change,ROC,No,0.002999
24,Relative Strength Index,RSI,No,0.011992
25,Inverse Fisher Transform RSI,IFT_RSI,No,0.026984
30,Bollinger Bands Width,BBWIDTH,No,0.008996
32,Percent B,PERCENT_B,No,0.009995
35,Directional Movement Indicator,DMI,No,0.121930
36,Average Directional Index,ADX,No,0.135944


##### Top 10 time consuming indicators

In [ ]:
indicators_analysis.sort_values(by='Processing time', ascending=False).head(7)

### Use Mlfinlab

In [ ]:
# import mlfinlab